In [15]:
# Part 1: Import Libraries and Define the Cage Class
import copy
import random

# from Demos.win32cred_demo import target


class Cage:
    def __init__(self, cells, operation, target):
        self.cells = cells
        self.operation = operation
        self.target = target
       


In [16]:
# Part 2: Function to Generate KenKen Puzzle
def generate_KenKen(size):
    
    grid = [[0] * size for _ in range(size)]
    fill_grid_with_numbers(grid,size)
    # for i in range(size):
    #     for j in range(size):
    #         print(str(grid[i][j])+' ',end=' ')
    #     print('\n')
    return  [[0] * size for _ in range(size)],generate_random_cages(grid,size)
    # return  grid,generate_random_cages(grid,size)


# Part 3: Function to Fill Grid with Numbers

def fill_grid_with_numbers(grid, size):
    """
    This function fills the grid with numbers 1 to size such that
    each row and each column has unique values. It uses backtracking
    to ensure all cells are filled with valid numbers.
    """
    

    def is_safe_to_place(grid, row, col, num):
        # Check row and column uniqueness
        
        for i in range(size) :
            if(i != col and grid[row][i]==num):
                return  False
            if(i != row and grid[i][col]==num):
                return  False
        return True
    
           


    def fill_grid_backtracking(grid, row, col):
        if row == size:
            return True  # If all rows are filled, the grid is complete

        # Move to the next row if we've filled all columns in the current row
        next_row = row + (col + 1) // size
        next_col = (col + 1) % size
        random_number = random.randint(1, size)
        for num in range(random_number,0,-1):
           if is_safe_to_place(grid, row, col, num):
                grid[row][col] = num  # Place the number
                if fill_grid_backtracking(grid, next_row, next_col):
                    return True  # Continue filling the next cells
                grid[row][col] = 0  # Backtrack (undo the placement)
        for num in range(random_number+1,size+1):
            if is_safe_to_place(grid, row, col, num):
                grid[row][col] = num  # Place the number
                if fill_grid_backtracking(grid, next_row, next_col):
                    return True  # Continue filling the next cells
                grid[row][col] = 0  # Backtrack (undo the placement)


        return False  # Return False if no valid number can be placed

                
       
    # Start backtracking from the first cell
    fill_grid_backtracking(grid, 0, 0)



# Part 4: Function to Generate Random Cages

def generate_random_cages(grid, size):
    cages = []
    visited = [[False] * size for _ in range(size)]

    for i in range(size):
        for j in range(size):
            if not visited[i][j]:
                cage_size = random.randint(1, size)
                cells = [(i, j)]
                visited[i][j] = True

                while len(cells) < cage_size:
                    x, y = cells[-1]
                    neighbors = [(x + dx, y + dy) for dx, dy in [(1, 0), (0, 1), (-1, 0), (0, -1)]
                                 if 0 <= x + dx < size and 0 <= y + dy < size and not visited[x + dx][y + dy]]
                    if neighbors:
                        next_cell = random.choice(neighbors)
                        cells.append(next_cell)
                        visited[next_cell[0]][next_cell[1]] = True
                    else:
                        break

                operation = random.choice(['+', '*']) if cage_size > 2 else random.choice(['+', '-', '*', '/'])
                if operation=='/' and len(cells)>1 and (grid[cells[0][0]][cells[0][1]]==0 or grid[cells[1][0]][cells[1][1]]==0):
                    operation = random.choice(['+', '-', '*'])
                target = calculate_target(grid, cells, operation)
                cages.append(Cage(cells, operation, target))

    return cages


# Function to calculate the target based on the operation for a given cage
def calculate_target(grid, cells, operation):
     if operation=='+':
       result = 0
       for x,y in cells:
           result+=grid[x][y]
       return  result
     
     elif operation=='*':
        result = 1
        for x,y in cells:
            result*=grid[x][y]
        return  result
     
     elif operation=='-':
          if(len(cells)>1):
            result = grid[cells[0][0]][cells[0][1]]
            result-=grid[cells[1][0]][cells[1][1]]
            return abs(result)
          else :
              return abs(grid[cells[0][0]][cells[0][1]])
     
     else:
         if(len(cells)>1):
            num1 = grid[cells[0][0]][cells[0][1]]
            num2 = grid[cells[1][0]][cells[1][1]]
            if num1>num2:return float(num1/num2)
            return  float(num2/num1)
         else :
              return float(grid[cells[0][0]][cells[0][1]])
             

In [17]:
# Part 5: KenKen Backtracking Solver
def solve_kenken(grid, cages):
    
    x,y = find_unassigned_location(grid)

    if x==-1:
       return True
        
    else:
        for i in range(1,len(grid)+1):
            if(is_safe_kenken(grid,x,y,i,cages)):
                 grid[x][y] = i
                 exist_flag=False
                 for cage in cages:
                     for cell in cage.cells:
                         if cell[0]==x and cell[1]==y:
                             exist_flag=True
                             break
                     if exist_flag==True:
                         values = [grid[x][y] for x,y in cage.cells]
                         if(validate_cage_operation(cage.operation,values,cage.target)):
                              if(solve_kenken(grid,cages)):
                                  return True
                         break
                       
                 grid[x][y]=0

    return False
        
            


def is_safe_kenken(grid, row, col, num, cages):
   # Check row and column numbers uniqueness

    for i in range(len(grid)):
        if(i!=col and grid[row][i]==num):
            return False
        if(i!=row and grid[i][col]==num):
            return  False
    return True


def validate_cage_operation(operation, values, target):
    # Check if the operation on the cage values matches the target
    if operation=='+':
      result=0
      for val in values:
          if val==0:
              return True
          result+=val
      return  True if result==target else False
    
    elif operation=='*':
        result = 1
        for val in values:
            if val==0:
                return True
            result*=val
        return True if result==target else False
    elif operation=='-':
        if(values[0]==0 or (len(values)==2 and values[1]==0)):
            return True
        result = abs(values[0]-values[1]) if len(values)>1 else abs(values[0])
        return True if result==target else False
    else:
        if(values[0]==0 or (len(values)==2 and values[1]==0)):
            return True
        result =values[0]
        if len(values)>1:
            num1 = values[0]
            num2 = values[1]
            if(min(num1,num2)==0):
                return False
            if num1>num2:
                result=float(num1/num2)
            else :
                result=float(num2/num1)
        return True if result==target else False
      
        

def find_unassigned_location(grid):
    for i in range(len(grid)):
        for j in range(len(grid[0])):
            if(grid[i][j]==0):
                return i,j
    return -1,-1



In [18]:
def solve_kenken_csp(grid, cages):

    def create_domains(grid):

        size = len(grid)
        domains = {}

        for i in range(size):
            for j in range(size):

                for cage in cages:
                    if cage.operation == '*' and (i, j) in cage.cells:
                        domains[(i, j)] = [i for i in range(1, size + 1) if cage.target % i == 0]
                        break
                    else:
                        domains[(i, j)] = list(range(1, size + 1))
        return domains

    def is_valid_assignment(i, j, val, assignment):

        if val not in assignment[(i, j)]:
            return False

        for cage in cages:
            if (i, j) in cage.cells:
                cage_values = [grid[row][col] for row, col in cage.cells if grid[row][col] != 0]
                if len(cage_values) == (len(cage.cells) - 1):
                    cage_values.append(val)
                    if not validate_cage_operation(cage.operation, cage_values, cage.target):
                        return False

        return True

    def find_unassigned_location():

        size = len(grid)
        for i in range(size):
            for j in range(size):
                if grid[i][j] == 0:
                    return i, j
        return None, None

    def forward_checking(i, j, val, assignment):

        size = len(grid)
        deleted = {}

        for k in range(size):
            if k != j and val in assignment[(i, k)]:
                assignment[(i, k)].remove(val)
                deleted[(i, k)] = val
            if k != i and val in assignment[(k, j)]:
                assignment[(k, j)].remove(val)
                deleted[(k, j)] = val
        return deleted

    def undo_changes(assignment, deleted):
        for cell, value in deleted.items():
            assignment[cell].append(value)

    def solve_csp(assignment):

        i, j = find_unassigned_location()
        if i is None:
            return True

        if len(assignment[(i, j)]) == 0:
            return False
        for val in assignment[(i, j)].copy():
            if is_valid_assignment(i, j, val, assignment):
                grid[i][j] = val
                deleted = forward_checking(i, j, val, assignment)
                if solve_csp(assignment):
                    return True
                undo_changes(assignment, deleted)
                grid[i][j] = 0

        return False

    domains = create_domains(grid)
    assignment = {(i, j): val for (i, j), val in domains.items()}

    if solve_csp(assignment):
        solved_grid = [[assignment.get((i, j)) for j in range(len(grid))] for i in range(len(grid))]
        solved_grid=grid
        return solved_grid
    else:
        return None


In [19]:
# Part 7: Print Solution

def print_solution(grid):
    for row in grid:
        print(" ".join(str(x) for x in row))
        


In [20]:
# Part 8: Run Example


def CheckFinal_Solution(grid,kenken_cages):
   '''   
   Just For Debugging Purpose

   Checks if the Final grid will Pass the Game Conditions or Not
   '''
   try : 
    for row in range(len(grid)):
        lst = [num for num in range(1,len(grid)+1)]
        for col in range(len(grid)):
            lst.remove(grid[row][col])
    for col in range(len(grid)):
        lst = [num for num in range(1,len(grid)+1)]
        for row in range(len(grid)):
            lst.remove(grid[row][col])
   except:
       return False
   
   for cage in kenken_cages:
       
       if cage.operation == '+':
           result = 0
           for row,col in cage.cells:
               result+=grid[row][col]
           if result!=cage.target:
               return False
       
       elif cage.operation=='-':
           if len(cage.cells)==1:
               if cage.target!=abs(grid[cage.cells[0][0]][cage.cells[0][1]]):
                   return False
           else:
               if cage.target!=abs(grid[cage.cells[0][0]][cage.cells[0][1]]-grid[cage.cells[1][0]][cage.cells[1][1]]):
                   return False
       
       elif cage.operation=='*':
           result = 1
           for row,col in cage.cells:
               result*=grid[row][col]
           if result!=cage.target:
               return False
       
       elif cage.operation=='/':
           if len(cage.cells)==1:
               if cage.target!=(grid[cage.cells[0][0]][cage.cells[0][1]]):
                   return False
           else :
               maxnum = max(grid[cage.cells[0][0]][cage.cells[0][1]],grid[cage.cells[1][0]][cage.cells[1][1]])
               minnum = min(grid[cage.cells[0][0]][cage.cells[0][1]],grid[cage.cells[1][0]][cage.cells[1][1]])
               if(maxnum/minnum!=cage.target):
                   return False
   return True



# size = 6
# unsolved_grid, kenken_cages = generate_KenKen(size)

# print("Generated KenKen Cages:")
# for cage in kenken_cages:
#     print(f"Cage Cells: {cage.cells}, Operation: {cage.operation}, Target: {cage.target}")
# print("-----------------------")

# Solve using Backtracking solver
# backtrack_grid = copy.deepcopy(unsolved_grid)
# if solve_kenken(backtrack_grid, kenken_cages):
#     print("Solved KenKen Puzzle (BackTracking):")
#     print(f"Status : { "ACCEPTED" if CheckFinal_Solution(backtrack_grid,kenken_cages) else "DENIED"}")
#     print_solution(backtrack_grid)

# else:
#     print("No solution found using BackTracking.")

# print("-----------------------")


# Solve using domain CSP solver
# csp_solved = solve_kenken_csp(unsolved_grid, kenken_cages)
# if csp_solved:
#     print("Solved KenKen Puzzle (domain CSP):")
#     print(f"Status : {"ACCEPTED" if CheckFinal_Solution(csp_solved,kenken_cages) else "DENIED"}")
#     print_solution(csp_solved)
# else:
#     print("No solution found using CSP.")


# back_failure = 0
# csp_failure = 0
# for i in range(100):
#     size = 5
#     unsolved_grid, kenken_cages = generate_KenKen(size)
    
#     # backtrack_grid = copy.deepcopy(unsolved_grid)
#     # if solve_kenken(backtrack_grid, kenken_cages):
      
#     #    back_failure+=not CheckFinal_Solution(backtrack_grid,kenken_cages)
    
#     csp_solved = solve_kenken_csp(unsolved_grid, kenken_cages)
#     if csp_solved:
#         csp_failure+=not CheckFinal_Solution(unsolved_grid,kenken_cages)

# print(f"back_failure : {back_failure}")
# print(f"csp_failure : {csp_failure}")




In [21]:
# Data Analysis
import time
import pandas as pd


def calculate_average_and_variance(data):
        n = len(data)
        if n == 0:
            return 0, 0  # Prevent division by zero
        
        # Calculate average
        average = sum(data) / n
        
        # Calculate variance
        variance = sum((x - average) ** 2 for x in data) / n
        
        return average, variance

experiment_data = {
     
     'Experiment': [],
     'Run_Number' : [],
    'Backtrack Average': [],
      'CSP Average': [],
    'Backtrack Variance': [],
    'CSP Variance': []

}

number_of_ex = 1

for experiment in range(number_of_ex):

    back_track_times = []
    csp_times = []

    start_time =0
    run_number = 100
    for i in range(run_number):
        size = 5
        unsolved_grid, kenken_cages = generate_KenKen(size)
        
        backtrack_grid = copy.deepcopy(unsolved_grid)
        start_time = time.time()
        solve_kenken(backtrack_grid, kenken_cages)
        back_track_times.append(time.time()-start_time)
        
        
        start_time = time.time()
        csp_solved = solve_kenken_csp(unsolved_grid, kenken_cages)
        csp_times.append(time.time()-start_time)


    


    # Calculate for Backtrack
    backtrack_average, backtrack_variance = calculate_average_and_variance(back_track_times)
   

    # Calculate for CSP
    csp_average, csp_variance = calculate_average_and_variance(csp_times)
    
    experiment_data['Experiment'].append(f'Experiment {experiment}')
    experiment_data['Run_Number'].append(run_number)
    experiment_data['Backtrack Average'].append(backtrack_average)
    experiment_data['Backtrack Variance'].append(backtrack_variance)
    experiment_data['CSP Average'].append(csp_average)
    experiment_data['CSP Variance'].append(csp_variance)




#  excel Table  : 




df = pd.DataFrame(experiment_data)

# Export to Excel
output_file = 'experiment_results_size5.xlsx'
df.to_excel(output_file, index=False, sheet_name='Results')

print(f"Data exported to {output_file} successfully.")





Data exported to experiment_results_size5.xlsx successfully.
